<a href="https://colab.research.google.com/github/madziejm/1e100-ibu/blob/master/1e100ibu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preliminary

#### Dependencies

In [1]:
import torch
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'dev = {dev}')

dev = cpu


In [2]:
!pip install --quiet icecream
from icecream import ic

In [ ]:
# !pip install --quiet -Iv torch==1.10.1
# !pip install --quiet -Iv torchtext==0.11.1

## Dataset representation

In [ ]:
!pip install 'spacy<3.3.0,>=3.2.0' --quiet
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 21.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
!pip show spacy | egrep Version
# we want SpaCy 3
!pip show torch | egrep Version
!pip show torchtext | egrep Version

Version: 3.2.1
Version: 1.10.0+cu111
Version: 0.11.0


#### review example

In [ ]:
#     """
#     beer/name: John Harvards Simcoe IPA
#     beer/beerId: 63836
#     beer/brewerId: 8481
#     beer/ABV: 5.4
#     beer/style: India Pale Ale &#40;IPA&#41;
#     review/appearance: 4/5
#     review/aroma: 6/10
#     review/palate: 3/5
#     review/taste: 6/10
#     review/overall: 13/20
#     review/time: 1157587200
#     review/profileName: hopdog
#     review/text: On tap at the Springfield, PA location. Poured a deep and cloudy orange (almost a copper) color with a small sized off white head. Aromas or oranges and all around citric. Tastes of oranges, light caramel and a very light grapefruit finish. I too would not believe the 80+ IBUs - I found this one to have a very light bitterness with a medium sweetness to it. Light lacing left on the glass.
#     """

#### dataset representation

In [3]:
from collections import Counter
from torchtext._torchtext import (Vocab as VocabPybind) # make use of some hidden interface
from torchtext.vocab import Vocab, build_vocab_from_iterator
from tqdm.notebook import trange, tqdm
import gc # garbage collector interface
import io
import re
import spacy # nlp toolkit
import torch

class RateBeerReviews(torch.utils.data.Dataset):
    def __init__(self, filepath='/content/SNAP-Ratebeer.txt', reviews_max=float('inf')):
        self.aspects = ['appearance', 'aroma', 'palate', 'taste', 'overall']
        self.aspect_count = len(self.aspects)
        self.aspect_max = [5, 10, 5, 10, 20]
        self._aspect_ratings = [ [] for _ in self.aspects ]
        self._texts = []
        self.unkn_tok = "<unk>" # unknown/out of vocabulary token
        self._len = 0
        self._fetch_data(filepath, reviews_max)
        self._post_process(max_word_count=20000) # 20K words should be okay

    def _fetch_data(self, filepath, reviews_max):
        with io.open(filepath, encoding='utf-8') as f:
            for line in tqdm(f, total=(40938282 if reviews_max == float('inf') else reviews_max * 14), desc='Reading data'):
                if line == '\n': # separator
                    self._len += 1
                    if reviews_max <= self._len:
                        break
                elif line.startswith('review/appearance: '):
                    line = line[len('review/appearance: '):]
                    self._aspect_ratings[0].append(int(line.split('/')[0])) # lhs of split by '/' is rating, rhs is max possible rating
                elif line.startswith('review/aroma: '):
                    line = line[len('review/aroma: '):]
                    self._aspect_ratings[1].append(int(line.split('/')[0])) # lhs of split by '/' is rating, rhs is max possible rating
                elif line.startswith('review/palate: '):
                    line = line[len('review/palate: '):]
                    self._aspect_ratings[2].append(int(line.split('/')[0])) # lhs of split by '/' is rating, rhs is max possible rating
                elif line.startswith('review/taste: '):
                    line = line[len('review/taste: '):]
                    self._aspect_ratings[3].append(int(line.split('/')[0])) # lhs of split by '/' is rating, rhs is max possible rating
                elif line.startswith('review/overall: '):
                    line = line[len('review/overall: '):]
                    self._aspect_ratings[4].append(int(line.split('/')[0])) # lhs of split by '/' is rating, rhs is max possible rating
                elif line.startswith('review/text: '):
                    line = line[len('review/text: '):]
                    if line.startswith('UPDATED:'):
                        line = line[len("UPDATED: APR 29, 2008"):] # drop prefix
                    line = re.sub('~', ' ', line.strip()) # remove whitespace incl. trailing newline and tildes that can be found in data for some reason
                    if line:
                        self._texts.append(line)
                    else: # some reviews do not have associated text; unwind (remove) their ratings for each aspect
                        for aspect_ratings in self._aspect_ratings:
                            aspect_ratings.pop()
                        self._len -= 1

    def _post_process(self, min_word_freq=None, max_word_count=None):
        assert (min_word_freq is not None) ^ bool(max_word_count is not None), "provide one of min_word_freq and max_word_count"
        nlp = spacy.util.get_lang_class('en')()
        nlp.add_pipe("sentencizer", config={"punct_chars": ['.', '?', '!']})
        nlp.Defaults.stop_words |= { '-', '+'}
        nlp.Defaults.stop_words -= {'mostly', 'whole', 'indeed', 'quite', 'ever', 'nothing', 'perhaps', 'not', 'no', 'only', 'well', 'really', 'except'}
        print("Spacy pipe (tokenization&sentence split)..")
        gc.collect() # force garbage collection
        self._texts = [tuple(list(tok.lower_ for tok in sent if not tok.is_stop and not tok.is_punct and not tok.is_space and len(tok) > 2) for sent in doc.sents if 0 != len(sent)) for doc in nlp.pipe(self._texts)]
        for i, text in enumerate(self._texts):
            assert 0 != len(text) # make sure no empty reviews again (new could be introduced by removing stop words unfortunately)
        print("Building vocab (word-id mapping)..")
        gc.collect() # force garbage collection
        sent_gen = (sent for text in self._texts for sent in text)
        if min_word_freq:
            self.vocab = build_vocab_from_iterator(sent_gen, specials=[self.unkn_tok], min_word_freq=5)
        else:
            words = Counter()
            for tokens in sent_gen:
                words.update(tokens)
            words = [word for word, freq in words.most_common(max_word_count)] # list sorted by frequency yikees
            self.vocab = Vocab(VocabPybind(words, None))
        self.vocab.insert_token(self.unkn_tok, 0)
        self.vocab.set_default_index(self.vocab[self.unkn_tok]) # set index for out-of-vocabulary words
        print("Mapping words to ids..")
        gc.collect() # force garbage collection
        self._texts = [tuple(self.vocab.lookup_indices(sent) for sent in text) for text in self._texts]
        gc.collect() # force garbage collection

    def __getitem__(self, i):
        # # 1 # python
        # sentences = tuple(sent for sent in self._texts[i])
        # ratings = tuple(self._aspect_ratings[a][i] for a in range(self.aspect_count))
        # 2 # tensor
        sentences = tuple(torch.LongTensor(sent) for sent in self._texts[i])
        ratings = torch.LongTensor(tuple(self._aspect_ratings[a][i] for a in range(self.aspect_count)))
        # # 3 # dev
        # sentences = tuple(torch.tensor(sent) for sent in self._texts[i])
        # ratings = torch.tensor(tuple(self._aspect_ratings[a][i] for a in range(self.aspect_count)))
        return (sentences, ratings)

    def __len__(self):
        return self._len

If you want to read dataset from dataset file, set FETCH_RATEBEER to true in the cell below and RECREATE_PICKLE to True. If you left them untouched, it'lle be read from serialized `RateBeerReviews` class object instead of parsing text file.

In [4]:
%%bash

export FETCH_RATEBEER=false
if [ "$FETCH_RATEBEER" = true ] && [ -e $RATEBEER_FILE ]
then # original dataset
    export RATEBEER_FILE='/content/SNAP-Ratebeer.txt'
    gdown --id '12tEEYQcHZtg5aWyfIiWWVIDAJNT-5d_T' # https://drive.google.com/file/d/12tEEYQcHZtg5aWyfIiWWVIDAJNT-5d_T/view?usp=sharing
    echo "Dataset head (trailing newline makes entry end): "
    head -n 16 $RATEBEER_FILE
    iconv -f ISO-8859-1 -t UTF-8 $RATEBEER_FILE -o {RATEBEER_FILE}.new && mv {RATEBEER_FILE}.new $RATEBEER_FILE
else # pickle
    gdown --id '1VBDjyR4jpzAgzcDUGNQFguOfLC3rtOV_' # https://drive.google.com/file/d/1VBDjyR4jpzAgzcDUGNQFguOfLC3rtOV_/view?usp=sharing  # 20K words dataset
    # gdown --id '1ebDMDlOxtFh8B5i8lajR7q3kq-0hM02j' # https://drive.google.com/file/d/1ebDMDlOxtFh8B5i8lajR7q3kq-0hM02j/view?usp=sharing # min frequency 5 words dataset
fi

Downloading...
From: https://drive.google.com/uc?id=1VBDjyR4jpzAgzcDUGNQFguOfLC3rtOV_
To: /content/ratebeer-20K-vocab.pickle
100%|##########| 393M/393M [00:02<00:00, 136MB/s]


In [5]:
import pickle # serialize lib
from google.colab import drive

drive.mount('/drive')

# DATASET_PICKLE='/content/ratebeer.pickle'
DATASET_PICKLE='/content/ratebeer-20K-vocab.pickle'
RECREATE_PICKLE = False

if RECREATE_PICKLE:
    rb = RateBeerReviews()
    with open(DATASET_PICKLE, 'wb') as f:
        print('Dumping..')
        torch.save(rb, f)
else:
    with open(DATASET_PICKLE, 'rb') as f:
        rb = torch.load(f)

Mounted at /drive


In [6]:
rb.aspect_max = [5, 10, 5, 10, 20] # hotfix
rb.anchor_words = { 'appearance' : ('appearance', 'color'), 
                    'aroma'      : ('aroma'),
                    'palate'     : ('palate', 'mouthfeel'),
                    'taste'      : ('taste'),
                    'overall'    : ('overall') }

In [ ]:
# word ID-s count and 1K of least common words
print(len(rb.vocab.get_itos()))
print(rb.vocab.get_itos()[-1000:])

20001
['tractor', 'syd', 'chiefly', 'defiant', 'stripper', 'val', 'bierhaus', 'eyeball', 'sporatic', 'deviate', 'grapenuts', 'beervana', 'tarness', 'lenny', 'phenomenon', 'reek', 'usualy', 'bernard', 'barbecued', 'fruitbeer', 'enjoyit', 'rive', 'sas', 'flavorable', 'ecstatic', 'leopoldstoch', 'soulless', 'allagashs', 'mikrobryggeri', 'disgusted', 'heine', 'covey', 'respectively', 'scout', 'emitting', 'flemdawg', 'atom', 'brightens', 'pch', 'www.bierzwerg.de', 'oakwood', 'flagged', 'collor', 'rbnag-11', 'belter', 'ecstasy', 'deem', 'gust', 'creamyish', 'opq', 'fruti', 'accompagne', 'zunge', 'sinful', 'prunish', 'ingrediants', 'refill', 'rug', 'alc%', 'blase', 'paramount', 'oldrtybastrd', 'fininsh', 'sofort', 'sazz', 'echoing', 'prospect', 'contributor', 'crazily', 'rogueone', 'beer_hawk', 'presenza', 'overlain', 'hinteren', 'spur', 'suspicions', 'glendale', 'renowned', 'hover', '09/08/2008', 'flashback', 'degraded', 'vegetabley', 'sunburst', 'snug', 'beneficial', 'linkery', 'mtn', 'doub

### Training (implementation of $(1)$)

In [19]:
from torch.utils.data import random_split, DataLoader
import datetime
from scipy.optimize import linear_sum_assignment
from more_itertools import grouper

# for wordcloudsdest_path=dest_path
import functools
import numpy as np
import os
import re
from PIL import Image
from os import path
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import functools
from operator import iadd

class Model():
    def __init__(self, dataset):
        self.ds = dataset
        self.init_weights()

    def init_weights(self):
        word_count = len(self.ds.vocab.get_itos())
        self.theta = torch.rand((word_count, self.ds.aspect_count), device=dev)
        # scale to [-0.1, 0.0], as we enforce this weight to 1.0 for some words later on
        self.theta = self.theta * -0.1
        # scale to [0.0, 0.9], as we enforce this weight to 1.0 for some words later on
        # self.theta = self.theta * 0.9
        # enforce 1 initialization on aspect name (page 4)
        # aspect_ids = self.ds.vocab.lookup_indices(self.ds.aspects)
        for aspect_idx, aspect in enumerate(self.ds.aspects):
            words = list(self.ds.anchor_words[aspect])

            words_ids = self.ds.vocab.lookup_indices(words)
            # ic(aspect_id)
            # ic(aspect_idx)
            # ic(self.ds.vocab.get_itos()[aspect_id])
            for word_id in words_ids:
                self.theta[word_id, aspect_idx] = 1.0
        self.theta.requires_grad_()

        # introduce separate phi for each aspect
        # self.phis = [torch.rand((word_count, self.ds.aspect_max[i])).to(dev) for i in range(self.ds.aspect_count)]
        self.phis = [torch.zeros((word_count, self.ds.aspect_max[i]), device=dev, dtype=self.theta.dtype) for i in range(self.ds.aspect_count)]
        # # normalize that sum across all words is 1 for a given aspect (eq. 7) # do not normalize as for now
        # self.phis = [phi / phi.sum(dim=0) for phi in self.phis]
        for phi in self.phis: phi.requires_grad_()
    
    def word_clouds(self, dest_path='/drive/MyDrive/Colab Notebooks/1e100ibu/saves/', filename='words.png', show=True):
        words = self.ds.vocab.get_itos()
        fig = plt.figure(figsize=(21, 5))
        plt.subplots_adjust(wspace=0.2)
        # plt.subplots_adjust(wspace=0.01, hspace=0.000000001)
        # plt.tight_layout()
        i = 0

        for aspect in range(self.ds.aspect_count):
            # print()
            aspect_name = self.ds.aspects[aspect]
            # print(aspect_name)
            atheta = self.theta[:, aspect].tolist()
            
            zipped = list(zip(words, atheta))
            # ic(list(filter(lambda x: x[0] == 'antrunk', zipped)))
            # sorted_zip = sorted(zipped, reverse=True, key=lambda x: x[1])[:50]
            # print(*[(word, '%s' % float('%.3g' % weight)) for (word, weight) in sorted_zip])

            wc = WordCloud(background_color="white", color_func=lambda *args, **kwargs: 'black', width=1000, height=1000)
            wc.generate_from_frequencies(dict(zipped))

            fig.add_subplot(self.ds.aspect_count, max(self.ds.aspect_max) + 1, i * (max(self.ds.aspect_max) + 1) + 1)
            plt.imshow(wc)
            plt.title(f'{aspect_name} Theta', fontsize=2)
            plt.axis("off")

            for rating in range(self.ds.aspect_max[aspect]):
                aphi = self.phis[aspect][:, rating].tolist()

                zipped = list(zip(words, aphi))
                sorted_zip = sorted(zipped, reverse=True, key=lambda x: x[1])[:200]

                if any(a == 0 for _, a in sorted_zip):
                    print(f'Omitting {aspect_name} Phi {str(rating + 1)}')
                    continue

                wc = WordCloud(background_color="white", color_func=lambda *args, **kwargs: 'black', width=1000, height=1000)
                wc.generate_from_frequencies(dict(zipped))
                fig.add_subplot(self.ds.aspect_count, max(self.ds.aspect_max) + 1, i * (max(self.ds.aspect_max) + 1) + rating + 2)

                plt.imshow(wc)
                plt.title(f'{aspect_name} Phi {str(rating + 1)}', fontsize=2)
                plt.axis("off")

            i += 1
        plt.savefig(f'{dest_path}/{filename}', dpi=600)
        plt.show(block=show)
    
    def rev_words_thetas(self, rev_sens_ids):
        """
        TODO filename sentence_aspects_likelihood_theta
        """
        return [self.theta[sen_ids] for sen_ids in rev_sens_ids]

    def rev_words_phis(self, rev_sens_ids):
        """
        TODO filename sentence_aspects_likelihood_phi
        """
        return [[self.phis[aspect_idx][sen_ids, :] for aspect_idx in range(self.ds.aspect_count)] for sen_ids in rev_sens_ids]
    
    def dump_weights(self, dest_path='/drive/MyDrive/Colab Notebooks/1e100ibu/saves/', filename=''):
        weights = {'phis': self.phis, 'theta': self.theta}
        torch.save(weights,  f'{dest_path}/{filename}')

    def load_weights(self, src_path):
        weights = torch.load(src_path, map_location=torch.device(dev))
        self.theta = weights['theta']
        self.phis  = weights['phis']
    
    def _linear_assignement(self, costs):
        # for nll we want to minimize
        return linear_sum_assignment(costs, maximize=False)
    
    def plot_nll_history(self, dir, filename):
        plt.figure().set_facecolor('white') # no alpha please
        plt.xlabel('epoch')
        plt.ylabel('NLL')
        plt.xlim(left=0.0, right=self.train_epoch_history[-1])
        plt.ylim(bottom=min(0.0, min(self.train_nll_history)), top=max(max(self.train_nll_history), max(self.test_nll_history)))
        plt.plot(self.train_epoch_history, self.train_nll_history, '-o')
        plt.plot(self.test_epoch_history, self.test_nll_history, '-o')
        plt.legend(['train batch NLL', 'test mean batch NLL'], loc='upper right')
        plt.title('Train and test dataset NLL')
        plt.savefig(os.path.join(dir, filename))
        plt.show()

    def experiments(self, root_dir='/drive/MyDrive/Colab Notebooks/1e100ibu/saves/'):
        num_of_runs = 3

        for idx in range(num_of_runs):
            directory = str(datetime.datetime.now())
            path = os.path.join(root_dir, directory)
            os.mkdir(path)

            self.train(dest_path=path)

        for idx in range(num_of_runs):
            directory = str(datetime.datetime.now()) + '-adam'
            path = os.path.join(root_dir, directory)
            os.mkdir(path)

            self.train(dest_path=path, optim='adam')

    def test_nll(self, debug=False):
        nlls = []
        with torch.no_grad():
            test_loader = DataLoader(self.test_ds, batch_size=100, collate_fn=lambda x: x) # do not use default collate function as it requires fixed-length input and raises this exception otherwise https://github.com/pytorch/pytorch/issues/42654
            for i, batch in enumerate(tqdm(test_loader, desc='test validation')):
                batch_nlls = []
                for (rev_sents_ids, review_aspects_scores) in batch:
                    rev_thetas = self.rev_words_thetas(rev_sents_ids)
                    rev_phis   = self.rev_words_phis(rev_sents_ids)
                    res_sents_scores = torch.stack(
                        [
                        rev_thetas[j].sum(dim=0) + torch.stack(tuple(rev_phis[j][a][:, review_aspects_scores[a] - 1].sum() for a in range(self.ds.aspect_count))) # 1 x aspect count
                        for j in range(len(rev_sents_ids))
                        ],
                    ) # sent count x aspect count

                    denoms = torch.logsumexp(res_sents_scores, dim=1)[:, None]
                    assert(denoms.shape == (len(rev_sents_ids), 1))

                    res_sents_scores = -res_sents_scores + denoms

                    sents_aspect_preds_max = torch.argmin(res_sents_scores, dim=1)
                    row_ind, col_ind = self._linear_assignement(costs=res_sents_scores.detach().cpu().numpy())
                    sents_aspect_preds_linear = sents_aspect_preds_max

                    sents_aspect_preds_linear[row_ind] = torch.from_numpy(col_ind).to(dev)
                    
                    batch_nlls.append(res_sents_scores.take_along_dim(sents_aspect_preds_linear[:, None], dim=1).sum().cpu().item())
                nlls.append(functools.reduce(lambda a, b: a + b, batch_nlls))
        nll_sum = functools.reduce(lambda a, b: a + b, nlls)
        if debug: ic(nll_sum)
        if debug: ic(torch.exp(-torch.tensor(nll_sum)))
        mean_batch_nll = nll_sum / len(nlls)
        return mean_batch_nll


    def train(self, dest_path='/drive/MyDrive/Colab Notebooks/1e100ibu/saves/', epoch_count=1, optim='sgd'):
        train_size = int(0.8 * len(self.ds))
        test_size = len(self.ds) - train_size

        params = (
            self.theta,
            *self.phis
        )
        lr = 0.0005
        # lr = sum(self.ds.aspect_max) * 0.01 / train_size # for whole batch iteration

        weight_decay = 0.00005
        momentum = 0.1
        if optim == 'sgd':
            self._optim = torch.optim.SGD(
                params=params,
                lr=lr,
                weight_decay=weight_decay,
                momentum=momentum
            )
        elif optim == 'adam': # do not use Adam, weights go to NaN with it for some reason
            self._optim = torch.optim.Adam(
                params=params,
                lr=lr,
                weight_decay=weight_decay,
                betas=(momentum, 0.999) # the first is params momentum, second RMSProp momentum (for now fix to 0.999 which is default Pytorch value)
            )
        else:
            assert False
        self._sched = torch.optim.lr_scheduler.StepLR(self._optim, step_size=1, gamma=0.95)


        self.train_ds, self.test_ds = random_split(self.ds, [train_size, test_size])

        train_loader = DataLoader(self.train_ds, batch_size=100, shuffle=True, collate_fn=lambda x: x) # do not use default collate function as it requires fixed-length input and raises this exception otherwise https://github.com/pytorch/pytorch/issues/42654
        self.train_epoch_history = []
        self.test_epoch_history = []
        self.train_nll_history = []
        self.test_nll_history = []
        try:
            for epoch in range(epoch_count):
                batch_count = len(train_loader)
                for i, batch in enumerate(tqdm(train_loader, desc=f'train epoch {epoch}/{epoch_count}')):
                    batch_nlls = []
                    for (rev_sents_ids, review_aspects_scores) in batch:
                        # for sent_ids in rev_sents_ids: sent_ids.to(dev)
                        rev_thetas = self.rev_words_thetas(rev_sents_ids)
                        rev_phis   = self.rev_words_phis(rev_sents_ids)

                        res_sents_scores = torch.stack(
                            [
                            rev_thetas[j].sum(dim=0) + torch.stack(tuple(rev_phis[j][a][:, review_aspects_scores[a] - 1].sum() for a in range(self.ds.aspect_count))) # 1 x aspect count
                            for j in range(len(rev_sents_ids))
                            ],
                        ) # sent count x aspect count

                        denoms = torch.logsumexp(res_sents_scores, dim=1)[:, None]
                        assert(denoms.shape == (len(rev_sents_ids), 1))

                        res_sents_scores = -res_sents_scores + denoms

                        sents_aspect_preds_max = torch.argmin(res_sents_scores, dim=1)
                        row_ind, col_ind = self._linear_assignement(costs=res_sents_scores.detach().cpu().numpy())
                        sents_aspect_preds_linear = sents_aspect_preds_max

                        # (most likely) aspect assignments (5)
                        sents_aspect_preds_linear[row_ind] = torch.from_numpy(col_ind).to(dev)
                        
                        # sentence likelihood (6)
                        batch_nlls.append(res_sents_scores.take_along_dim(sents_aspect_preds_linear[:, None], dim=1).sum())

                    batch_nll = torch.stack(batch_nlls).sum()
                    self._optim.zero_grad(set_to_none=True)
                    if 0 == i % 100:
                        self.train_nll_history.append(batch_nll.cpu().detach().item())
                        self.train_epoch_history.append(epoch + i / batch_count)
                    if 0 == i % 5000:
                        if i != 0:
                            self.dump_weights(dest_path=dest_path, filename=(f'-epoch-{epoch}-{epoch_count}-{int(i)}'))
                            self.word_clouds(dest_path=dest_path, filename=(f'cloud-epoch-{epoch}-{epoch_count}-{int(i)}.png'))
                            ic(self.rev_words_thetas([self.ds.vocab.lookup_indices(['taste', 'aroma', 'palete', 'antrunk'])]))
                        # self.test_nll_history.append(self.test_nll())
                        # self.test_epoch_history.append(epoch + i / batch_count)
                    batch_nll.backward()

                    self._optim.step()
                self._sched.step()

            self.dump_weights(dest_path=dest_path, filename=(f'weights-end'))
            self.word_clouds(dest_path=dest_path, filename=(f'cloud-end.png'))
            self.plot_nll_history(dest_path, (f'plot-end.png'))

        except KeyboardInterrupt:
            print('Interrupted.')

            self.dump_weights(dest_path=dest_path, filename=(f'weights-interrupt'))
            self.word_clouds(dest_path=dest_path, filename=(f'cloud-interrupt.png'))
            self.plot_nll_history(dest_path, (f'plot-interrupt.png'))

        except Exception as e:
            ic(len(batch_nlls))
            ic(denoms)
            ic(rev_sents_ids)

            self.dump_weights(dest_path=dest_path, filename=(f'weights-exception'))
            self.word_clouds(dest_path=dest_path, filename=(f'cloud-exception.png'))
            self.plot_nll_history(dest_path, (f'plot-exception.png'))

            ic(batch_nll)
            ic(denoms.size())
            ic((len(rev_sents_ids),))
            ic(denoms.size() == (len(rev_sents_ids),))
            raise e


In [20]:
model = Model(rb)

In [ ]:
%%capture output
# %%script python --no-raise-error
# model.train()
model.experiments(root_dir='./saves/')

In [ ]:
output.show()

In [ ]:
# %store output > output_log
import pickle
with open('output', 'wb') as f:
    pickle.dump(output, f)

In [ ]:
model.word_clouds()

In [ ]:
indices = model.ds.vocab.lookup_indices(['soul', 'tyskie', '1/3', 'antrunk', 'nett', '<unk>'])
# indices = model.ds.vocab.lookup_indices(['antrunk'])Napi

In [ ]:
# model.train()
ic(self.rev_words_thetas([self.ds.vocab.lookup_indices(['taste', 'aroma', '1/3', 'antrunk', 'nett', '<unk>'])]))

# weights = torch.tensor((2.0, 1.0, 3.0, 7.0))
# # sorted_zip = sorted(zipped, reverse=True, key=lambda x: x[1])[:50]
# sorted(list(zip('j p g m'.split(), weights)), reverse=True, key=lambda x: x[1])[:2]


[tensor([[ 6.2895e-02,  7.7352e-02,  3.8608e-01,  7.7248e-01,  4.9354e-01],
         [ 6.1379e-03,  1.1089e-01,  1.4684e-01,  1.5803e+00, -2.0832e-01],
         [ 3.9038e-01,  1.8007e-01,  3.8165e-01, -8.4364e-02,  5.1858e-01],
         [ 6.5582e+00, -1.8309e+00, -2.0364e-01, -1.4185e+00, -1.1770e+00],
         [-1.2600e-01,  1.3653e-01,  1.1414e+00,  3.4624e-01,  2.3008e-01],
         [ 6.1379e-03,  1.1089e-01,  1.4684e-01,  1.5803e+00, -2.0832e-01]],
        grad_fn=<IndexBackward0>)]

In [ ]:
itos = model.ds.vocab.get_itos()

In [ ]:
for i, (rev_sents_ids, review_aspects_scores) in enumerate(model.test_ds):
    if i > 9:
        break
    print(list(zip(model.ds.aspects, review_aspects_scores.tolist())))
    for sen_ids in rev_sents_ids:
        print([itos[idx] for idx in sen_ids])
        # print(model.ds.vocab.lookup_indices(.tolist()))
    print(rev_sents_ids, review_aspects_scores)
    aspects = model.ided_review_forward(rev_sents_ids, review_aspects_scores)
    print([model.ds.aspects[aspect] for aspect in aspects])
    print()

[('appearance', 4), ('aroma', 7), ('palate', 3), ('taste', 8), ('overall', 15)]
['<unk>', 'world', 'live', '<unk>', '2008']
['amber', 'beer', 'good', 'head']
['aroma', 'malty', 'rich', 'flavour', 'rich', 'vinous', 'malty']
['overall', 'nice', 'vinous', 'rich', 'beer']
(tensor([   0,  602, 1256,    0,  490]), tensor([27,  5, 14,  1]), tensor([  2,  28, 102,  62, 102, 474,  28]), tensor([ 86,  11, 474, 102,   5])) tensor([ 4,  7,  3,  8, 15])
['overall', 'aroma', 'palate', 'overall']

[('appearance', 3), ('aroma', 8), ('palate', 4), ('taste', 8), ('overall', 16)]
['pours', 'light', 'amber']
['great', 'american', 'hop', 'aroma', 'tones', 'pine', 'grapefruit', 'tangerine', 'flowers', 'apricot']
['sweet', 'malt', 'background']
['subtle', 'tropical', 'tones', 'present']
['spicy', 'hop', 'flavour', 'tones', 'grapefruit', 'pine', 'apricot', 'tangerine']
['fruity']
['discrete', 'malty', 'background']
['light', 'sweet', 'medium', 'bitter']
['medium', 'dry']
['medium', 'bodied']
['favourite', 'ip

In [ ]:
# danger
# model.ds.vocab.lookup_indices([0])

## Word clouds

In [ ]:
pip install wordcloud